<a href="https://colab.research.google.com/github/0xD4rky/Sparks-Foundation/blob/main/Convolve_2_Mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/TrainingDataBank.csv')

<ipython-input-293-d69168735915>:1: DtypeWarning: Columns (5,18,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/TrainingDataBank.csv')


In [ ]:
df_val = pd.read_csv('/content/Bank_Validation_Data.csv')

<ipython-input-294-cbe4bd292ec6>:1: DtypeWarning: Columns (4,17,22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_val = pd.read_csv('/content/Bank_Validation_Data.csv')


In [ ]:
df.head(5)

,Primary key,Target,account_opening_date,country_code,demog_1,demog_2,income,demog_3,city_tier,occupation,...,demog_39,email_domain,demog_40,demog_41,others_42,others_43,others_44,others_45,demog_42,demog_43
0,1,0,2/18/2023,IN,53.0,3.0,100001 to 5L,101.0,Rural,Self_Employed,...,1,gmail,low,1,NaN,0.0,NaN,NaN,1,medium
1,2,0,2/1/2023,IN,55.0,1.0,0 to 1L,101.0,Tier 1,Salaried,...,1,gmail,High,1,NaN,NaN,NaN,NaN,1,High
2,3,0,12/9/2022,IN,55.0,3.0,0 to 1L,101.0,Tier 2,Self_Employed,...,1,gmail,medium,1,NaN,NaN,NaN,NaN,1,High
3,4,0,3/31/2023,IN,53.0,3.0,100001 to 5L,101.0,Tier 7,Self_Employed,...,1,gmail,High,1,NaN,NaN,NaN,NaN,1,High
4,5,0,1/17/2023,IN,55.0,6.0,0 to 1L,101.0,Tier 1,Student,...,1,gmail,low,1,NaN,NaN,NaN,NaN,1,medium


In [ ]:
df_val.head(5)

,Primary key,account_opening_date,country_code,demog_1,demog_2,income,demog_3,city_tier,occupation,demog_4,...,demog_39,email_domain,demog_40,demog_41,others_42,others_43,others_44,others_45,demog_42,demog_43
0,9000001,3/25/2023,IN,56.0,2.0,5L to 10L,101.0,Tier 7,Self_Employed,N,...,1,gmail,High,1,NaN,NaN,NaN,NaN,1,High
1,9000002,1/15/2023,IN,55.0,1.0,100001 to 5L,101.0,Tier 5,Salaried,N,...,1,gmail,low,1,NaN,NaN,NaN,NaN,1,medium
2,9000003,1/6/2023,IN,55.0,1.0,100001 to 5L,101.0,Tier 1,Other,N,...,1,gmail,low,1,NaN,NaN,NaN,NaN,1,medium
3,9000004,2/8/2023,IN,55.0,1.0,100001 to 5L,101.0,Tier 1,Other,N,...,1,gmail,medium,1,NaN,0.0,NaN,NaN,1,High
4,9000005,1/27/2023,IN,55.0,1.0,100001 to 5L,101.0,Tier 5,Other,N,...,1,gmail,low,1,NaN,NaN,NaN,NaN,1,High


In [ ]:
unique_value_counts_df_main = df.nunique()
columns_with_nunique_1 = unique_value_counts_df_main[unique_value_counts_df_main == 1].index.tolist()
# List of columns to be removed
columns_to_remove = ['demog_7', 'demog_10', 'demog_12', 'txn_35', 'txn_36', 'txn_49', 'txn_50', 'txn_65', 'txn_70', 'txn_71', 'txn_72', 'demog_38','others_42','others_43','others_44','others_45','country_code']

# Remove columns from the original DataFrame
df = df.drop(columns=columns_to_remove)

# code of conversion from date to numeric data
df['account_opening_date'] = pd.to_datetime(df['account_opening_date'], format='%m/%d/%Y')

# Extract day, month, and year into new columns
df['opening_day'] = df['account_opening_date'].dt.day
df['opening_month'] = df['account_opening_date'].dt.month
df['opening_year'] = df['account_opening_date'].dt.year

# Drop the original 'account_opening_date' column
df = df.drop('account_opening_date', axis=1)

# Deleting rows
df = df.dropna(subset=['occupation'])
# Resetting the index after dropping rows
df.reset_index(drop=True, inplace=True)
df = df.dropna(subset=['city_tier'])
# Resetting the index after dropping rows
df.reset_index(drop=True, inplace=True)
# Filter rows where 'demog_2' column has 'ZZ' value and indices are in the specified list
rows_to_delete = df[df['demog_2'] == 'ZZ']

# Delete the filtered rows from the DataFrame
df.drop(rows_to_delete.index, inplace=True)

# Replace missing values in 'income' column with '100001 to 5L' where 'Target' is equal to 1
# df.loc[df['Target'] == 1, 'income'].fillna('100001 to 5L', inplace=True)

# Set missing values in 'demog_40' column to 'High' where 'Target' is equal to 1
df.loc[df['Target'] == 1, 'demog_40'].fillna('High', inplace=True)

#Filling the missing value of demog_4 column
df['demog_4'].fillna('N', inplace=True)

# Create a binary column indicating the presence of 'N'
df['is_N'] = df['demog_4'].apply(lambda x: 1 if x == 'N' else 0)

# Create a new numeric column for the numeric values
df['numeric_values_demog_4'] = pd.to_numeric(df['demog_4'], errors='coerce')

# Fill 0 where 'N' was present
df['numeric_values_demog_4'].fillna(0, inplace=True)
df = df.drop(columns=['demog_4'])

df['income'].fillna('100001 to 5L', inplace=True)

<ipython-input-297-ccbb5d1eda70>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(rows_to_delete.index, inplace=True)
<ipython-input-297-ccbb5d1eda70>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['demog_4'].fillna('N', inplace=True)
<ipython-input-297-ccbb5d1eda70>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_N'] = df['demog_4'].apply(lambda x: 1 if x

In [ ]:
unique_value_counts_df_val_main = df_val.nunique()
columns_with_nunique_1 = unique_value_counts_df_val_main[unique_value_counts_df_val_main == 1].index.tolist()
# List of columns to be removed
columns_to_remove = ['demog_7', 'demog_10', 'demog_12', 'txn_35', 'txn_36', 'txn_49', 'txn_50', 'txn_65', 'txn_70', 'txn_71', 'txn_72', 'demog_38','others_42','others_43','others_44','others_45','country_code']

# Remove columns from the original DataFrame
df_val = df_val.drop(columns=columns_to_remove)

# code of conversion from date to numeric data
df_val['account_opening_date'] = pd.to_datetime(df_val['account_opening_date'], format='%m/%d/%Y')

# Extract day, month, and year into new columns
df_val['opening_day'] = df_val['account_opening_date'].dt.day
df_val['opening_month'] = df_val['account_opening_date'].dt.month
df_val['opening_year'] = df_val['account_opening_date'].dt.year

# Drop the original 'account_opening_date' column
df_val = df_val.drop('account_opening_date', axis=1)

# Deleting rows
df_val = df_val.dropna(subset=['occupation'])
# Resetting the index after dropping rows
df_val.reset_index(drop=True, inplace=True)
df_val = df_val.dropna(subset=['city_tier'])
# Resetting the index after dropping rows
df_val.reset_index(drop=True, inplace=True)
# Filter rows where 'demog_2' column has 'ZZ' value and indices are in the specified list
rows_to_delete = df_val[df_val['demog_2'] == 'ZZ']

# Delete the filtered rows from the DataFrame
df_val.drop(rows_to_delete.index, inplace=True)

# Replace missing values in 'income' column with '100001 to 5L' where 'Target' is equal to 1
# df_val.loc[df_val['Target'] == 1, 'income'].fillna('100001 to 5L', inplace=True)

# Set missing values in 'demog_40' column to 'High' where 'Target' is equal to 1
df_val['demog_40'].fillna('High', inplace=True)

#Filling the missing value of demog_4 column
df_val['demog_4'].fillna('N', inplace=True)

# Create a binary column indicating the presence of 'N'
df_val['is_N'] = df_val['demog_4'].apply(lambda x: 1 if x == 'N' else 0)

# Create a new numeric column for the numeric values
df_val['numeric_values_demog_4'] = pd.to_numeric(df_val['demog_4'], errors='coerce')

# Fill 0 where 'N' was present
df_val['numeric_values_demog_4'].fillna(0, inplace=True)
df_val = df_val.drop(columns=['demog_4'])

df_val['income'].fillna('100001 to 5L', inplace=True)

In [ ]:
# Assuming df is your DataFrame

# Fill missing values in 'os' column with 'and'
df['os'].fillna('and', inplace=True)

# Fill missing values in 'demog_13' column with True
df.loc[:, 'demog_13'].fillna(True, inplace=True)
df['demog_13'] = df['demog_13'].astype(bool)

# Fill missing values in 'demog_14' column with False
df.loc[:, 'demog_14'].fillna(False, inplace=True)
df['demog_14'] = df['demog_14'].astype(bool)

# Fill missing values in 'demog_15' column with True
df.loc[:, 'demog_15'].fillna(True, inplace=True)
df['demog_15'] = df['demog_15'].astype(bool)

# Fill missing values in 'demog_16' column with True
df.loc[:, 'demog_16'].fillna(True, inplace=True)
df['demog_16'] = df['demog_16'].astype(bool)

# Fill missing values in 'demog_9' column with False
df.loc[:, 'demog_9'].fillna(False, inplace=True)
df['demog_9'] = df['demog_9'].astype(bool)

# Fill missing values in 'demog_17' column with True
df.loc[:, 'demog_17'].fillna(True, inplace=True)
df['demog_17'] = df['demog_17'].astype(bool)

# Fill missing values in 'demog_18' column with False
df.loc[:, 'demog_18'].fillna(False, inplace=True)
df['demog_18'] = df['demog_18'].astype(bool)

# Fill missing values in 'demog_19' column with True
df.loc[:, 'demog_19'].fillna(True, inplace=True)
df['demog_19'] = df['demog_19'].astype(bool)

# Fill missing values in 'demog_20' column with False
df.loc[:, 'demog_20'].fillna(False, inplace=True)
df['demog_20'] = df['demog_20'].astype(bool)

# Fill missing values in 'demog_21' column with False
df.loc[:, 'demog_21'].fillna(False, inplace=True)
df['demog_21'] = df['demog_21'].astype(bool)

df['demog_2'].fillna(1.0, inplace=True)

boolean_columns = ['demog_17', 'demog_18', 'demog_19', 'demog_20', 'demog_21','demog_9','demog_13','demog_14','demog_15','demog_16']

# Convert boolean columns to numerical values (0 for False, 1 for True)
df[boolean_columns] = df[boolean_columns].astype(int)

df['demog_30'].fillna(0, inplace=True)
df['demog_31'].fillna(0, inplace=True)
df['demog_32'].fillna(0, inplace=True)
df['demog_33'].fillna(1, inplace=True)
df['demog_34'].fillna(1, inplace=True)
df['demog_35'].fillna(0, inplace=True)
df['demog_36'].fillna(0, inplace=True)
df['demog_37'].fillna(0, inplace=True)
df['demog_3'].fillna(101, inplace=True)

In [ ]:
# Assuming df_val is your DataFrame

# Fill missing values in 'os' column with 'and'
df_val['os'].fillna('and', inplace=True)

# Fill missing values in 'demog_13' column with True
df_val.loc[:, 'demog_13'].fillna(True, inplace=True)
df_val['demog_13'] = df_val['demog_13'].astype(bool)

# Fill missing values in 'demog_14' column with False
df_val.loc[:, 'demog_14'].fillna(False, inplace=True)
df_val['demog_14'] = df_val['demog_14'].astype(bool)

# Fill missing values in 'demog_15' column with True
df_val.loc[:, 'demog_15'].fillna(True, inplace=True)
df_val['demog_15'] = df_val['demog_15'].astype(bool)

# Fill missing values in 'demog_16' column with True
df_val.loc[:, 'demog_16'].fillna(True, inplace=True)
df_val['demog_16'] = df_val['demog_16'].astype(bool)

# Fill missing values in 'demog_9' column with False
df_val.loc[:, 'demog_9'].fillna(False, inplace=True)
df_val['demog_9'] = df_val['demog_9'].astype(bool)

# Fill missing values in 'demog_17' column with True
df_val.loc[:, 'demog_17'].fillna(True, inplace=True)
df_val['demog_17'] = df_val['demog_17'].astype(bool)

# Fill missing values in 'demog_18' column with False
df_val.loc[:, 'demog_18'].fillna(False, inplace=True)
df_val['demog_18'] = df_val['demog_18'].astype(bool)

# Fill missing values in 'demog_19' column with True
df_val.loc[:, 'demog_19'].fillna(True, inplace=True)
df_val['demog_19'] = df_val['demog_19'].astype(bool)

# Fill missing values in 'demog_20' column with False
df_val.loc[:, 'demog_20'].fillna(False, inplace=True)
df_val['demog_20'] = df_val['demog_20'].astype(bool)

# Fill missing values in 'demog_21' column with False
df_val.loc[:, 'demog_21'].fillna(False, inplace=True)
df_val['demog_21'] = df_val['demog_21'].astype(bool)

df_val['demog_2'].fillna(1.0, inplace=True)

boolean_columns = ['demog_17', 'demog_18', 'demog_19', 'demog_20', 'demog_21','demog_9','demog_13','demog_14','demog_15','demog_16']

# Convert boolean columns to numerical values (0 for False, 1 for True)
df_val[boolean_columns] = df_val[boolean_columns].astype(int)

df_val['demog_30'].fillna(0, inplace=True)
df_val['demog_31'].fillna(0, inplace=True)
df_val['demog_32'].fillna(0, inplace=True)
df_val['demog_33'].fillna(1, inplace=True)
df_val['demog_34'].fillna(1, inplace=True)
df_val['demog_35'].fillna(0, inplace=True)
df_val['demog_36'].fillna(0, inplace=True)
df_val['demog_37'].fillna(0, inplace=True)
df_val['demog_3'].fillna(101, inplace=True)

In [ ]:
def find_rows_with_missing_values(df, column_name):
    rows_with_missing_values = df[df[column_name].isna()]
    return rows_with_missing_values.index

In [ ]:
from sklearn.impute import SimpleImputer

# Assuming df is your DataFrame
# Create an instance of SimpleImputer with strategy='most_frequent'
imputer = SimpleImputer(strategy='most_frequent')

df['demog_40'] = imputer.fit_transform(df[['demog_40']])
df['demog_43'] = imputer.fit_transform(df[['demog_43']])
df['demog_8'] = imputer.fit_transform(df[['demog_8']])
df['demog_22'] = imputer.fit_transform(df[['demog_22']])
# If you have multiple columns with missing values, you can use the following
# df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [ ]:
from sklearn.impute import SimpleImputer

# Assuming df_val is your DataFrame
# Create an instance of SimpleImputer with strategy='most_frequent'
imputer = SimpleImputer(strategy='most_frequent')

df_val['demog_40'] = imputer.fit_transform(df_val[['demog_40']])
df_val['demog_43'] = imputer.fit_transform(df_val[['demog_43']])
df_val['demog_8'] = imputer.fit_transform(df_val[['demog_8']])
df_val['demog_22'] = imputer.fit_transform(df_val[['demog_22']])
# If you have multiple columns with missing values, you can use the following
# df_val = pd.DataFrame(imputer.fit_transform(df_val), columns=df_val.columns)

In [ ]:
column_names = df.columns.tolist()

In [ ]:
column_names_val = df_val.columns.tolist()

In [ ]:
def analyze_column(df, column_name):
    # Value counts where 'Target' is 0
    value_counts_target_0 = df[df['Target'] == 0][column_name].value_counts()

    # Value counts where 'Target' is 1
    value_counts_target_1 = df[df['Target'] == 1][column_name].value_counts()

    # Null values where 'Target' is 0
    null_values_target_0 = df[df['Target'] == 0][column_name].isnull().sum()

    # Null values where 'Target' is 1
    null_values_target_1 = df[df['Target'] == 1][column_name].isnull().sum()

    return value_counts_target_0, value_counts_target_1, null_values_target_0, null_values_target_1

In [ ]:
from sklearn.impute import KNNImputer

# Assuming df is your DataFrame
# Extract the relevant columns for imputation
impute_columns = ['demog_24','demog_26','demog_1']
# Create a copy of the DataFrame to avoid modifying the original data
df_impute = df.copy()

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Perform imputation on the selected columns
df_impute[impute_columns] = knn_imputer.fit_transform(df_impute[impute_columns])

# Update the original DataFrame with the imputed values
df[impute_columns] = df_impute[impute_columns]

In [ ]:
from sklearn.impute import KNNImputer

# Assuming df_val is your DataFrame
# Extract the relevant columns for imputation
impute_columns = ['demog_24','demog_26','demog_1']
# Create a copy of the DataFrame to avoid modifying the original data
df_val_impute = df_val.copy()

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Perform imputation on the selected columns
df_val_impute[impute_columns] = knn_imputer.fit_transform(df_val_impute[impute_columns])

# Update the original DataFrame with the imputed values
df_val[impute_columns] = df_val_impute[impute_columns]

In [ ]:
# Extract the relevant columns for imputation
impute_columns1 = ['demog_5']

# Create a copy of the DataFrame to avoid modifying the original data
df_impute1 = df.copy()

# Initialize the KNNImputer
knn_imputer1 = KNNImputer(n_neighbors=1)

# Perform imputation on the selected columns
df_impute1[impute_columns1] = knn_imputer1.fit_transform(df_impute1[impute_columns1])

# Update the original DataFrame with the imputed values
df[impute_columns1] = df_impute1[impute_columns1]

In [ ]:
# Extract the relevant columns for imputation
impute_columns1 = ['demog_5']

# Create a copy of the DataFrame to avoid modifying the original data
df_val_impute1 = df_val.copy()

# Initialize the KNNImputer
knn_imputer1 = KNNImputer(n_neighbors=1)

# Perform imputation on the selected columns
df_val_impute1[impute_columns1] = knn_imputer1.fit_transform(df_val_impute1[impute_columns1])

# Update the original DataFrame with the imputed values
df_val[impute_columns1] = df_val_impute1[impute_columns1]

In [ ]:
demog_prefix = 'demog_'
txn_prefix = 'txn_'
others_prefix = 'others_'

# Identify columns with demographic attributes
demog_columns = [col for col in df.columns if col.startswith(demog_prefix)]
df_demog = df[demog_columns]

# Identify columns with transaction attributes
txn_columns = [col for col in df.columns if col.startswith(txn_prefix)]
df_tx = df[txn_columns]

# Identify columns with other attributes
others_columns = [col for col in df.columns if col.startswith(others_prefix)]
df_others = df[others_columns]

df_object = df.select_dtypes(include=['object'])

# Making column list of all different dataframes
df_tx_column_names_list = df_tx.columns.tolist()
df_demog_column_names_list = df_demog.columns.tolist()
df_others_column_names_list = df_others.columns.tolist()
column_names_of_df_object = df_object.columns.tolist()

In [ ]:
demog_prefix = 'demog_'
txn_prefix = 'txn_'
others_prefix = 'others_'

# Identify columns with demographic attributes
demog_columns = [col for col in df_val.columns if col.startswith(demog_prefix)]
df_val_demog = df_val[demog_columns]

# Identify columns with transaction attributes
txn_columns = [col for col in df_val.columns if col.startswith(txn_prefix)]
df_val_tx = df_val[txn_columns]

# Identify columns with other attributes
others_columns = [col for col in df_val.columns if col.startswith(others_prefix)]
df_val_others = df_val[others_columns]

df_val_object = df_val.select_dtypes(include=['object'])

# Making column list of all different dataframes
df_val_tx_column_names_list = df_val_tx.columns.tolist()
df_val_demog_column_names_list = df_val_demog.columns.tolist()
df_val_others_column_names_list = df_val_others.columns.tolist()
column_names_of_df_val_object = df_val_object.columns.tolist()

In [ ]:
# List of columns to exclude with object data type
exclude_columns = ['account_opening_date', 'country_code', 'demog_2', 'income', 'city_tier', 'occupation',
                   'demog_4', 'demog_17', 'demog_18', 'demog_19', 'demog_20', 'demog_21', 'demog_22',
                   'os', 'email_domain', 'demog_40', 'demog_43']

# Select only numerical columns by excluding the specified columns
numerical_df = df.select_dtypes(include=['number']).drop(columns=exclude_columns, errors='ignore')

In [ ]:
x = find_rows_with_missing_values(df,'txn_1').tolist()
tx_missing_rows_rows = ['txn_1']
count = 0
for i in range(1,73):
  if (find_rows_with_missing_values(df,df_tx_column_names_list[i]).tolist()==x):
    tx_missing_rows_rows.append(df_tx_column_names_list[i])
    count = count + 1
  x = find_rows_with_missing_values(df,df_tx_column_names_list[i]).tolist()

In [ ]:
target_counts = df['Target'].value_counts()
target_counts

0    93747
1     1999
Name: Target, dtype: int64

In [ ]:
unique_value_counts = df_tx.nunique()

In [ ]:
unique_value_counts

txn_1      63
txn_2      88
txn_3      50
txn_4      74
txn_5     303
         ... 
txn_77     90
txn_78     30
txn_79     90
txn_80    799
txn_81    344
Length: 73, dtype: int64

In [ ]:
distinct_values_counts_details = df_tx.apply(lambda x: pd.Series({'Distinct Values': x.value_counts().index.tolist(), 'Count': x.value_counts().tolist()})).T

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95746 entries, 0 to 95748
Columns: 164 entries, Primary key to numeric_values_demog_4
dtypes: float64(123), int64(32), object(9)
memory usage: 120.5+ MB


In [ ]:
data_types = df.dtypes

In [ ]:
data_types.value_counts()

float64    123
int64       32
object       9
dtype: int64

In [ ]:
df_object = df.select_dtypes(include=['object'])

# Print or use the new DataFrame
print("New DataFrame with 'object' Data Type Columns:")
df_object

New DataFrame with 'object' Data Type Columns:


,demog_2,income,city_tier,occupation,demog_22,os,email_domain,demog_40,demog_43
0,3.0,100001 to 5L,Rural,Self_Employed,Y,ios,gmail,low,medium
1,1.0,0 to 1L,Tier 1,Salaried,Y,and,gmail,High,High
2,3.0,0 to 1L,Tier 2,Self_Employed,Y,and,gmail,medium,High
3,3.0,100001 to 5L,Tier 7,Self_Employed,Y,and,gmail,High,High
4,6.0,0 to 1L,Tier 1,Student,Y,and,gmail,low,medium
...,...,...,...,...,...,...,...,...,...
95744,2,0 to 1L,Rural,Self_Employed,Y,and,gmail,High,High
95745,6,100001 to 5L,Rural,Student,Y,and,gmail,low,medium
95746,1,100001 to 5L,Tier 6,Salaried,Y,and,gmail,High,medium
95747,1,5L to 10L,Tier 1,Other,Y,ios,gmail,High,High


In [ ]:
column_names_of_df_object = df_object.columns.tolist()

In [ ]:
column_names_of_df_object

['demog_2',
 'income',
 'city_tier',
 'occupation',
 'demog_22',
 'os',
 'email_domain',
 'demog_40',
 'demog_43']

In [ ]:
df.iloc[:, 5].value_counts() #mixed data type

101.0    92251
103.0     2167
102.0      590
104.0      501
213.0      222
168.0        8
105.0        4
106.0        3
Name: demog_3, dtype: int64

In [ ]:
df.isnull().mean()*100

Primary key               0.0
Target                    0.0
demog_1                   0.0
demog_2                   0.0
income                    0.0
                         ... 
opening_day               0.0
opening_month             0.0
opening_year              0.0
is_N                      0.0
numeric_values_demog_4    0.0
Length: 164, dtype: float64

In [ ]:
unique_value_counts_df_main = pd.DataFrame(df.nunique())

In [ ]:
df_demog.isnull().mean()*100

demog_1     0.0
demog_2     0.0
demog_3     0.0
demog_5     0.0
demog_6     0.0
demog_8     0.0
demog_9     0.0
demog_11    0.0
demog_13    0.0
demog_14    0.0
demog_15    0.0
demog_16    0.0
demog_17    0.0
demog_18    0.0
demog_19    0.0
demog_20    0.0
demog_21    0.0
demog_22    0.0
demog_23    0.0
demog_24    0.0
demog_25    0.0
demog_26    0.0
demog_27    0.0
demog_28    0.0
demog_29    0.0
demog_30    0.0
demog_31    0.0
demog_32    0.0
demog_33    0.0
demog_34    0.0
demog_35    0.0
demog_36    0.0
demog_37    0.0
demog_39    0.0
demog_40    0.0
demog_41    0.0
demog_42    0.0
demog_43    0.0
dtype: float64

In [ ]:
df_others.isnull().mean()*100

others_1      0.000000
others_2      0.000000
others_3      0.001044
others_4      0.001044
others_5     13.644434
others_6      0.003133
others_7      0.003133
others_8      0.003133
others_9      0.003133
others_10     0.003133
others_11     0.003133
others_12     0.000000
others_13     0.000000
others_14     0.000000
others_15    39.072128
others_16    39.072128
others_17    39.072128
others_18     0.000000
others_19     0.001044
others_20     0.001044
others_21    14.133227
others_22    14.133227
others_23    14.133227
others_24    14.133227
others_25    14.133227
others_26    14.133227
others_27     0.009400
others_28     0.009400
others_29     0.001044
others_30     0.001044
others_31     0.001044
others_32     0.001044
others_33     0.000000
others_34     0.000000
others_35     0.000000
others_36     0.000000
others_37     0.000000
others_38     0.000000
others_39     0.000000
others_40     0.001044
others_41     0.001044
dtype: float64

In [ ]:
df_object.isnull().mean()*100

demog_2         0.0
income          0.0
city_tier       0.0
occupation      0.0
demog_22        0.0
os              0.0
email_domain    0.0
demog_40        0.0
demog_43        0.0
dtype: float64

In [ ]:
analyze_column(df,'city_tier')

(Tier 1    25323
 Rural     18885
 Tier 8    11804
 Tier 7    10179
 Tier 6     8766
 Tier 3     6200
 Tier 5     5533
 Tier 2     4817
 Tier 4     2240
 Name: city_tier, dtype: int64,
 Rural     538
 Tier 1    354
 Tier 8    274
 Tier 6    211
 Tier 7    209
 Tier 2    127
 Tier 3    126
 Tier 5     96
 Tier 4     64
 Name: city_tier, dtype: int64,
 0,
 0)

In [ ]:
# Encoding dataframe starts from here , ordinal and One-Hot encoding.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
# City Tier column
# Define the order for encoding
tier_order = ['Rural', 'Tier 8', 'Tier 7', 'Tier 6', 'Tier 5', 'Tier 4', 'Tier 3', 'Tier 2', 'Tier 1']

# Create an instance of OrdinalEncoder with the specified order
encoder = OrdinalEncoder(categories=[tier_order])

# Fit and transform the 'city_tier' column
df['city_tier_encoded'] = encoder.fit_transform(df[['city_tier']])

# Drop the original 'city_tier' column
df = df.drop(columns=['city_tier'])
df.rename(columns={'city_tier_encoded': 'city_tier'}, inplace=True)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
# City Tier column
# Define the order for encoding
tier_order = ['Rural', 'Tier 8', 'Tier 7', 'Tier 6', 'Tier 5', 'Tier 4', 'Tier 3', 'Tier 2', 'Tier 1']

# Create an instance of OrdinalEncoder with the specified order
encoder = OrdinalEncoder(categories=[tier_order])

# Fit and transform the 'city_tier' column
df_val['city_tier_encoded'] = encoder.fit_transform(df_val[['city_tier']])

# Drop the original 'city_tier' column
df_val = df_val.drop(columns=['city_tier'])
df_val.rename(columns={'city_tier_encoded': 'city_tier'}, inplace=True)

In [ ]:
# Define the mapping for ordinal encoding
ordinal_mapping = {'High': 3, 'medium': 2, 'low': 1}

# Create an instance of OrdinalEncoder with the specified mapping
encoder = OrdinalEncoder(categories=[sorted(ordinal_mapping, key=ordinal_mapping.get)])

# Apply ordinal encoding to 'demog_40' and 'demog_43'
df['demog_40'] = encoder.fit_transform(df[['demog_40']])
df['demog_43'] = encoder.fit_transform(df[['demog_43']])

In [ ]:
# Define the mapping for ordinal encoding
ordinal_mapping = {'High': 3, 'medium': 2, 'low': 1}

# Create an instance of OrdinalEncoder with the specified mapping
encoder = OrdinalEncoder(categories=[sorted(ordinal_mapping, key=ordinal_mapping.get)])

# Apply ordinal encoding to 'demog_40' and 'demog_43'
df_val['demog_40'] = encoder.fit_transform(df_val[['demog_40']])
df_val['demog_43'] = encoder.fit_transform(df_val[['demog_43']])

In [ ]:
# Assuming df is your DataFrame and 'income' is the column to be encoded
income_mapping = {
    '0': 0,
    '0 to 1L': 1,
    '100001 to 5L': 2,
    '5L to 10L': 3,
    '10L to 25L': 4,
    '25L to 50L': 5,
    '50L to 3Crore': 6,
    '3Crore to 5Crore': 7,
    '5Crore to 10Crore': 8,
    '10Crore to 15Crore': 9,
    '15Crore to 20Crore': 10,
    '20Crore to 35Crore': 11
}

df['income'] = df['income'].map(income_mapping)

In [ ]:
# Assuming df_val is your DataFrame and 'income' is the column to be encoded
income_mapping = {
    '0': 0,
    '0 to 1L': 1,
    '100001 to 5L': 2,
    '5L to 10L': 3,
    '10L to 25L': 4,
    '25L to 50L': 5,
    '50L to 3Crore': 6,
    '3Crore to 5Crore': 7,
    '5Crore to 10Crore': 8,
    '10Crore to 15Crore': 9,
    '15Crore to 20Crore': 10,
    '20Crore to 35Crore': 11
}

df_val['income'] = df_val['income'].map(income_mapping)

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# # Assuming df is your DataFrame and 'occupation' is the column to be one-hot encoded
# encoder = OneHotEncoder(drop='first',sparse_output=False)

# # Reshape the data to a 2D array
# occupation_encoded = encoder.fit_transform(df[['occupation']])

# # Create a DataFrame from the one-hot encoded array
# occupation_ohe = pd.DataFrame(occupation_encoded, columns=encoder.get_feature_names_out(['occupation']))

# # Concatenate the one-hot encoded columns with the original DataFrame
# df = pd.concat([df, occupation_ohe], axis=1)

# # Drop the original 'occupation' column
# df.drop('occupation', axis=1, inplace=True)

In [ ]:
df1 = df.copy()

In [ ]:
df2 = df_val.copy()

In [ ]:
import pandas as pd

# Assuming 'occupation' is the column you want to one-hot encode
# df is your DataFrame

# Use get_dummies to one-hot encode the 'occupation' column
df_encoded = pd.get_dummies(df, columns=['occupation'], drop_first=True)

# If you want to replace the original 'occupation' column with the encoded columns
# Uncomment the next line
df = pd.concat([df.drop(['occupation'], axis=1), df_encoded], axis=1)

In [ ]:
# Assuming 'occupation' is the column you want to one-hot encode
# df_val is your DataFrame

# Use get_dummies to one-hot encode the 'occupation' column
df_val_encoded = pd.get_dummies(df_val, columns=['occupation'], drop_first=True)

# If you want to replace the original 'occupation' column with the encoded columns
# Uncomment the next line
df_val = pd.concat([df_val.drop(['occupation'], axis=1), df_val_encoded], axis=1)

In [ ]:
analyze_column(df1,'os')

KeyError: 'emial_domain'